In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

## Additional RAM

In [ ]:
# But it is no more proposed get more RAM in a free way
[1]*10**10

## data

In [29]:
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"
ratings = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv"

In [30]:
#Cleaned data base with scrapped books
df_books = pd.read_csv(books, sep = "|")

In [31]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [32]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [33]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [64]:
df_ratings[df_ratings.ISBN == "000648302X"]

,User-ID,ISBN,Book-Rating
45384,11676,000648302X,8
174443,37950,000648302X,9
373708,89624,000648302X,9
412327,98544,000648302X,0
417691,99328,000648302X,3
496961,119654,000648302X,8
499864,120610,000648302X,7
654880,158589,000648302X,8
659263,160016,000648302X,4
718973,174092,000648302X,0


## Tests merge

In [ ]:
DF_CatUsers = pd.DataFrame( {"UserID": [1,1,1,2,2,3,3,3,3], 
                             "ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN2", "ISBN3", "ISBN5", "ISBN6"], 
                             "Ratings": [4,5,3,2,1,4,5,4,4]})
DF_CatUsers

In [ ]:
DF_CatBooks = pd.DataFrame( {"ISBN": ["ISBN1", "ISBN2", "ISBN3", "ISBN4", "ISBN5", "ISBN6", "ISBN7", "ISBN8"], 
                             "features": ["Blabla1", "Blabla2", "Blabla3", "Blabla4", "Blabla5", "Blabla6", "Blabla7", "Blabla8"]})
DF_CatBooks

In [ ]:
pd.merge(DF_CatUsers, DF_CatBooks, on = "ISBN")

## Merge

In [34]:
#Join of ratings and books data base, according to ISBN which is a common column
df_bert = pd.merge(df_ratings, df_books, left_on = "ISBN", right_on = "isbn")  
len(df_bert)

110801

In [35]:
#I will only embeds lines with a not empty book_description
lineOK = df_bert[df_bert["book_description"].isna() == False]
len(lineOK)

75976

In [36]:
#Creation of an additional column in order to put the future book_description embedding
df_bert = df_bert.assign(description_bert = 0)

In [37]:
lineOK.index[997:1002]

Int64Index([1549, 1550, 1551, 1552, 1553], dtype='int64')

In [38]:
len(df_bert)

110801

In [95]:
df_bert.head()

,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,0


In [96]:
len(df_bert)

110801

## Test descriptions

In [ ]:
for d in df_books["book_description"][:3]:
    print(d)

## Bert

### Loading

In [39]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.7MB/s 
     |████████████████████████████████| 901kB 53.1MB/s 
     |████████████████████████████████| 3.3MB 54.2MB/s 


In [40]:
from transformers import BertTokenizer, BertModel

In [41]:
#I will use an already trained BERT network
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Tokenizer

In [49]:
df_bert.book_description[100].split(".")

['This gripping tale unfolds in a peaceful Minnesota town, where crime is something that just doesn&#39;t happen',
 ' But when a young boy disappears, it marks the beginning of a unspeakable nightmare',
 '']

In [97]:
#execution in 1min 30sec

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for i in lineOK.index:
    
    sent = df_bert.book_description[i]

    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.

    encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                  )
        
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
        
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [98]:
print(len(input_ids))
print(len(input_ids[0]))

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
input_ids.size()

75976
1


torch.Size([75976, 64])

In [ ]:
input_ids[:10000].size()

In [99]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_bert.book_description[0])
print('Token IDs:', input_ids[0])

Original:  Twenty years after having an affair with legendary artist George Gabriel as a college student, Genny Haviland is accused of drugging and suffocating the painter, despite her claim that she only did it to ease her former lover to a painless ...
Token IDs: tensor([  101,  3174,  2086,  2044,  2383,  2019,  6771,  2007,  8987,  3063,
         2577,  6127,  2004,  1037,  2267,  3076,  1010,  8991,  4890,  5292,
        14762,  5685,  2003,  5496,  1997,  4319,  4726,  1998, 10514,  4246,
        27483,  1996,  5276,  1010,  2750,  2014,  4366,  2008,  2016,  2069,
         2106,  2009,  2000,  7496,  2014,  2280,  7089,  2000,  1037,  3255,
         3238,  1012,  1012,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [100]:
input_ids[500]

tensor([  101,  4895, 24597, 19779, 23808,  2000,  2014,  3129,  1010,  1037,
         2160, 19993,  7009,  1037,  3117,  2104,  1037, 13881,  1998,  2009,
         4150,  1037, 24304,  1012,  2014,  3129,  4515,  2039,  2437,  1037,
         3185,  1997,  2009,  1010,  2145,  2025,  4209,  2008,  2010,  2564,
         2626,  1996,  2338,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])

### Model use

In [19]:
#Use of pretrained model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [20]:
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [21]:
#With use of GPU I will save lots of time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [23]:
#with only 5 000 lines, it crash, because it "needs more RAM"

# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(input_ids[:1000].to(device), attention_masks[:1000].to(device))

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]


### Tests concat outputs

In [40]:
hs = hidden_states

In [41]:
token_embeddings = torch.stack(hs, dim=0)
print(token_embeddings.size())

token_embeddings = token_embeddings.permute(1,2,0,3)
print(token_embeddings.size())

torch.cat((token_embeddings, token_embeddings), 0)

torch.Size([13, 1000, 64, 768])
torch.Size([1000, 64, 13, 768])


RuntimeError: ignored

### Output dim

In [24]:
print("device:", device)

#Outputs
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

device: cuda:0
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1000
Number of tokens: 64
Number of hidden units: 768


In [25]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())


      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1000, 64, 768])


In [26]:
#Current dimensions: [# layers, # batches, # tokens, # features]
#Desired dimensions: [# tokens, # layers, # features]

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)
print(token_embeddings.size())

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,2,0,3)
print(token_embeddings.size())

torch.Size([13, 1000, 64, 768])
torch.Size([1000, 64, 13, 768])


### Sentence encoding

In [27]:
#Or create a sentence embeddings
# `hidden_states` has shape [13, 1000, 64, 768]

# `token_vecs` is a tensor with shape [1000, 64, 768]
token_vecs = hidden_states[-2]
print(token_vecs.size())

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=1)
print(sentence_embedding.size())

torch.Size([1000, 64, 768])
torch.Size([1000, 768])


### Insertion in dataframe

In [28]:
sentence_embedding = sentence_embedding.cpu().numpy()

In [29]:
sentence_embedding.shape

(1000, 768)

In [56]:
ind_emb = 0
for ind_df in lineOK.index[:1000]:
  df_bert.description_bert[ind_df] = [str(sentence_embedding[ind_emb][i]) for i in range(len(sentence_embedding[0]))]
  ind_emb = ind_emb + 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [57]:
print(len(df_bert))
df_bert[997:1002]

75976


,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
1549,39467,8408011200,0,8408011200,0.0,UOM:39015032487327,El hermano pequeño,Manuel Vazquez Montalban,1994-01-01,Planeta Publishing Corporation,literary critic,NaN,Enthält: El hermano pequeno ; La soledad acomp...,es,http://books.google.com/books/content?id=VmdfA...,221.0,fiction,NaN,"[https://www.goodreads.com/book/show/9671587, ...",NaN,NaN,"[-0.28538537, -0.21242969, -0.15521047, 0.3636..."
1550,77039,8408011200,0,8408011200,0.0,UOM:39015032487327,El hermano pequeño,Manuel Vazquez Montalban,1994-01-01,Planeta Publishing Corporation,literary critic,NaN,Enthält: El hermano pequeno ; La soledad acomp...,es,http://books.google.com/books/content?id=VmdfA...,221.0,fiction,NaN,"[https://www.goodreads.com/book/show/9671587, ...",NaN,NaN,"[-0.28538537, -0.21242969, -0.15521047, 0.3636..."
1551,95902,8408011200,7,8408011200,0.0,UOM:39015032487327,El hermano pequeño,Manuel Vazquez Montalban,1994-01-01,Planeta Publishing Corporation,literary critic,NaN,Enthält: El hermano pequeno ; La soledad acomp...,es,http://books.google.com/books/content?id=VmdfA...,221.0,fiction,NaN,"[https://www.goodreads.com/book/show/9671587, ...",NaN,NaN,"[-0.28538537, -0.21242969, -0.15521047, 0.3636..."
1552,153079,8408011200,0,8408011200,0.0,UOM:39015032487327,El hermano pequeño,Manuel Vazquez Montalban,1994-01-01,Planeta Publishing Corporation,literary critic,NaN,Enthält: El hermano pequeno ; La soledad acomp...,es,http://books.google.com/books/content?id=VmdfA...,221.0,fiction,NaN,"[https://www.goodreads.com/book/show/9671587, ...",NaN,NaN,0
1553,178534,8408011200,0,8408011200,0.0,UOM:39015032487327,El hermano pequeño,Manuel Vazquez Montalban,1994-01-01,Planeta Publishing Corporation,literary critic,NaN,Enthält: El hermano pequeno ; La soledad acomp...,es,http://books.google.com/books/content?id=VmdfA...,221.0,fiction,NaN,"[https://www.goodreads.com/book/show/9671587, ...",NaN,NaN,0


In [87]:
len(sentence_embedding[0])

768

In [94]:
type(sentence_embedding[0])

numpy.ndarray

In [105]:
sentence_embedding[0][:5]

array([-0.18947688,  0.02510934,  0.19813403, -0.4776927 ,  0.48648745],
      dtype=float32)

In [97]:
str(sentence_embedding[0][0])

'-0.18947688'

In [98]:
float(str(sentence_embedding[0][0]))

-0.18947688

In [102]:
l = [str(sentence_embedding[0][i]) for i in range(len(sentence_embedding[0]))]
len(l)

768

In [104]:
l[:5]

['-0.18947688', '0.025109343', '0.19813403', '-0.4776927', '0.48648745']

### Automatisation

In [101]:
#It takes about 8min

for i_1000 in range(75):
  with torch.no_grad():

      outputs = model(input_ids[(i_1000*1000):((i_1000+1)*1000)].to(device), attention_masks[(i_1000*1000):((i_1000+1)*1000)].to(device))
      hidden_states = outputs[2]

      # `hidden_states` has shape [13, 1000, 64, 768] 
      # => hidden_states[-2] has shape [1000, 64, 768] 
      token_vecs = hidden_states[-2]
      #sentence_embedding has shape [1000, 768] : 
      #=> mean elements by elements of each token (64 tokens in total)
      sentence_embedding = torch.mean(token_vecs, dim=1)

      sentence_embedding = sentence_embedding.cpu().numpy()

      ind_emb = 0
      for ind_df in lineOK.index[(i_1000*1000):((i_1000+1)*1000)]:
        df_bert.description_bert[ind_df] = [str(sentence_embedding[ind_emb][i]) for i in range(len(sentence_embedding[0]))]
        ind_emb = ind_emb + 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [102]:
len(df_bert)

110801

In [104]:
df_bert.to_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|", index = False)

# Train / Test data creation

In [4]:
#We get the bert embedding of book_description 
df_bert = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")

In [5]:
#We only use the not empty book_description lines
lineOK = df_bert[df_bert["book_description"].isna() == False][df_bert["Book-Rating"] != 0] 
len(lineOK)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


29932

In [83]:
print(len(df_bert))
df_bert.head()

110801


,User-ID,ISBN,Book-Rating,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_bert
0,276725,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
1,2313,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
2,6543,034545104X,0,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
3,8680,034545104X,5,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."
4,10314,034545104X,9,034545104X,0.0,0.0,Flesh Tones : A Novel,M. J. Rose,2002-01-01,NaN,fiction,NaN,Twenty years after having an affair with legen...,en,http://books.google.com/books/content?id=3MrPj...,310.0,literatur,fiction,[],NaN,NaN,"['-0.18947688', '0.025109343', '0.19813403', '..."


In [7]:
X_data = df_bert[["User-ID", "description_bert"]]
X_data.head()

,User-ID,description_bert
0,276725,"['-0.18947688', '0.025109343', '0.19813403', '..."
1,2313,"['-0.18947688', '0.025109343', '0.19813403', '..."
2,6543,"['-0.18947688', '0.025109343', '0.19813403', '..."
3,8680,"['-0.18947688', '0.025109343', '0.19813403', '..."
4,10314,"['-0.18947688', '0.025109343', '0.19813403', '..."


In [8]:
Y_data = df_bert["Book-Rating"]
Y_data.head()

0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [9]:
len(X_data)

110801

In [10]:
MIN = min(df_bert["User-ID"])
MAX = max(df_bert["User-ID"])
print("min:", MIN)
print("max:", MAX)

min: 8
max: 278854


In [11]:
#Transformation of a X_data line into a vector 
#beacause bert embedding has been saved as a list of strings
def vectBert(l):
  #bert embedding of dim768 and User-ID
  b = np.zeros(768+1)

  #1st copy of User-ID
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #THen handling of list of strings
  v = X_data.iloc[0][1].split("'")
  b[1:] = [float(v[i]) for i in range(1, len(v), 2)]
  
  return b

In [12]:
print(len(vectBert(X_data.iloc[0])))
vectBert(X_data.iloc[0])[0:5]

769


array([ 0.99236496, -0.18947688,  0.02510934,  0.19813403, -0.4776927 ])

In [13]:
#treatment of all lines of X_data such that X_data.book_description is not empty
def matBert(l, df):
  m = np.zeros((len(l), (768+1)))
  
  ind = 0
  for i in l.index:
      m[ind] = vectBert(df.iloc[i])
      ind = ind + 1
  
  return m

In [14]:
#It takes about 1min for about 75 000 lines
mat_X_data = matBert(lineOK, X_data)
mat_X_data.shape

(29932, 769)

In [15]:
#treatment of all corresponding lines of Y_data
#=> same lines as lines for which X_data.book_description is not empty
def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [16]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(29932, 1)

In [17]:
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [18]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 29932 

X_train.shape (23945, 769)
X_test.shape (5987, 769) 

y_train.shape (23945, 1)
y_test.shape (5987, 1)


In [129]:
type(X_train)

numpy.ndarray

In [130]:
X_train[:5]

array([[ 0.19897004, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.24817641, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.55151589, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.93762148, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486],
       [ 0.49270565, -0.18947688,  0.02510934, ..., -0.07602315,
         0.05378568, -0.02498486]])

In [65]:
y_train[0]

array([0])

# Ordinal regression

### Model training

In [19]:
!pip install mord
from mord.threshold_based import LogisticAT

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=f06adbb6edb600f4555f21692c50acc049433733a576710459ac31750da36d98
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


In [20]:
LogisticAT().get_params().keys()

dict_keys(['alpha', 'max_iter', 'verbose'])

In [21]:
np.logspace(3, 5, 5)

array([  1000.        ,   3162.27766017,  10000.        ,  31622.77660168,
       100000.        ])

In [146]:
dico_param = {
    'max_iter': np.logspace(3, 5, 5).astype(int),
    'alpha': np.logspace(0, 4, 5)
}

recherche_hyper = GridSearchCV(estimator = LogisticAT(), \
			param_grid = dico_param, scoring="accuracy", cv=5)

In [124]:
Counter(np.squeeze(y_train))

Counter({1: 106,
         2: 161,
         3: 358,
         4: 514,
         5: 2616,
         6: 2026,
         7: 4246,
         8: 5818,
         9: 3643,
         10: 4457})

In [23]:
LAT = LogisticAT(max_iter = 10000)
LAT.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticAT(alpha=1.0, max_iter=10000, verbose=0)

### Model validation

In [24]:
Counter(np.squeeze(y_test))

Counter({1: 26,
         2: 40,
         3: 90,
         4: 129,
         5: 654,
         6: 506,
         7: 1062,
         8: 1455,
         9: 911,
         10: 1114})

In [25]:
#always predicts the most represented class !
y_pred = LAT.predict(X_test)

In [26]:
y_pred.shape

(5987,)

In [27]:
Counter(y_pred)

Counter({8: 5987})

In [ ]:
model.save("/content/drive/MyDrive/ProjetML/OrdinalRegression.model")